## 파파고 api를 사용한 Back Translation Augmentation

In [1]:
import pandas as pd
import time
import os
import sys
import urllib.request

In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../../data')

In [5]:
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))

In [ ]:
target = 'text'

#### 파파고 api 사용 가이드 https://developers.naver.com/docs/papago/papago-nmt-overview.md
##### api key를 직접 생성 후 입력 필요

In [ ]:
# api key 10개를 사용
client_id = ["","","","","","","","","",""]
client_secret = ["","","","","","","","","",""]
limit = 10000
use = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
def ToEN(koText, client_id, client_secret):
    encText = urllib.parse.quote(koText)
    data = "source=ko&target=en&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        return eval(response_body.decode('utf-8'))['message']['result']['translatedText']
    else:
        print("Error Code:" + rescode)

In [ ]:
def ToKO(egText, client_id, client_secret):
    kocText = urllib.parse.quote(egText)
    data = "source=en&target=ko&text=" + kocText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        return eval(response_body.decode('utf-8'))['message']['result']['translatedText']
    else:
        print("Error Code:" + rescode)

#### 사용할 index 범위를 지정

In [ ]:
idx_start = 0
idx_end = 100

##### 사용량을 측정해 자동으로 api key를 변경
##### 모든 할당량을 초과하면 자동 정지

In [ ]:
arrKO = []
total_use = [0] * len(client_id)
i = 0
for idx, s in enumerate(data[target][idx_start: idx_end+1]):
    start = time.time()
    if len(s) > limit - use[i] - total_use[i]:
        i+=1
        if i>=len(client_id):
            print('out of usage')
            break
        print('key change to',i)
        print(total_use)
        
    total_use[i] += len(s)
    tmp = ToEN(s,client_id[i],client_secret[i])
    
    if len(tmp) > limit - use[i] - total_use[i]:
        i+=1
        if i>=len(client_id):
            print('out of usage')
            break
        print('key change to',i)
        print(total_use)
    total_use[i] += len(tmp)
    
    tmp = ToKO(tmp, client_id[i], client_secret[i])
    arrKO.append(tmp)
    
    end = time.time()
    print(f"{idx_start+idx} {end-start:.5f} sec")

In [ ]:
copy = data.iloc[idx_start:idx_end+1, :]
copy[target]=arrKO
copy

In [ ]:
copy.to_csv(f'papago/idx{idx_start}-{idx_end}.csv',index=False)

## papago 폴더의 csv 파일들 합치기

In [ ]:
folder_path = "papago"
file_list = [file for file in os.listdir(folder_path) if file.endswith(".csv")]
sorted_files = sorted(file_list, key=lambda x: int(x.split("idx")[1].split("-")[0]))
sorted_files

In [ ]:
dfs = []
for file in sorted_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

In [ ]:
combined_df = pd.concat(dfs, ignore_index=True)
combined_df

In [ ]:
combined_df.to_csv('papagoTrans.csv',index=False)